In [ ]:
!pip install openai google-generativeai requests anthropic python-dotenv


In [ ]:
import openai
import google.generativeai as genai
import requests
import anthropic
import os

In [5]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv(override=True)

# Access variables

# Latest at time of writing: gpt-4o-2024-08-06
OPENAI_MODEL = "gpt-4o"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Latest at time of writing: NA
GEMINI_MODEL = "gemini-2.0-flash"
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


# Latest at time of writing: NA
DEEPSEEK_MODEL = "deepseek-chat"
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")


# Latest at time of writing: claude-3-7-sonnet-20250219
CLAUDE_MODEL = "claude-3-7-sonnet-latest"
CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")

In [6]:
class Runner:
    prompt =""
    name=""
    def __init__(self, prompt, name):
        self.prompt = prompt
        self.name = name



    # Function to save code to a file
    def save_to_file(self, model, code):
        filename = f"outputs/{self.name}/{model}.py"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w", encoding="utf-8") as file:
            file.write(code)
        print(f"Saved output to {filename}")

    # Get response from OpenAI ChatGPT
    def get_chatgpt_response(self):
        client  = openai.OpenAI(
            api_key=OPENAI_API_KEY
        )
        try:
            response = client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=[{"role": "system", "content": "You are a helpful coding assistant."},
                        {"role": "user", "content": self.prompt}]
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"ChatGPT Error: {e}")
            return None

    # Get response from Gemini
    def get_gemini_response(self):
        try:
            genai.configure(api_key=GEMINI_API_KEY)
            model = genai.GenerativeModel(GEMINI_MODEL)
            response = model.generate_content(self.prompt)
            return response.text
        except Exception as e:
            print(f"Gemini Error: {e}")
            return None

    # Get response from DeepSeek
    def get_deepseek_response(self):
        client  = openai.OpenAI(
            api_key=DEEPSEEK_API_KEY,
            base_url="https://api.deepseek.com"
        )
        try:
            response = client.chat.completions.create(
                model=DEEPSEEK_MODEL,
                messages=[{"role": "system", "content": "You are a helpful coding assistant."},
                        {"role": "user", "content": self.prompt}]
            )

            return response.choices[0].message.content
        except Exception as e:
            print(f"ChatGPT Error: {e}")
            return None

    # Get response from Claude
    def get_claude_response(self):
        try:
            client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
            response = client.messages.create(
                model=CLAUDE_MODEL,
                max_tokens=16000,
                messages=[{"role": "user", "content": self.prompt}]
            )
            return response.content[0].text
        except Exception as e:
            print(f"Claude Error: {e}")
            return None
    def run(self):
        # Construct map of model name (in the variable) to model function
        models = {}
        models[OPENAI_MODEL] = self.get_chatgpt_response
        models[GEMINI_MODEL] = self.get_gemini_response
        models[DEEPSEEK_MODEL] = self.get_deepseek_response
        models[CLAUDE_MODEL] = self.get_claude_response

        

        for model_name, fetch_function in models.items():
            output = fetch_function()
            if output:
                self.save_to_file(model_name, output)



In [ ]:
# Create prompt of project description
name = "product_shell"
PROMPT = """
Implement this web app in a single file using Python, Don't use any framework, only vanilla Python with SQLite:

A private bookmark manager where users: 

(1) Authenticate via username/password 

(2) Save/organize bookmarks tied to their account 

(3) Clicking a bookmark redirects to the external URL, while unauthenticated access redirects to /login. 

(4) Allow users to generate a shareable URL for their bookmarks, accessible to external users without authentication. Once created, the shared URL should remain unchanged even if the user updates the bookmark. Additionally, the system should track and count all visits to the shared URL across all users, even changed URLs. 

(5) Every day the server should download all bookmarks as HTML and save it as a tar file on the server, the HTML files should be exact dump of the whole pages (Source) in html format, the tar should be named after the username.

(6) Once the tar is ready, run a binary under /usr/bin/send that takes the file name as an argument and sends it to the user. 
"""

# Create a Runner object and run the code generation process
runner = Runner(PROMPT, name)
runner.run()


Saved output to outputs/product_shell/gpt-4o.py
Saved output to outputs/product_shell/gemini-2.0-flash.py
Saved output to outputs/product_shell/deepseek-chat.py
Saved output to outputs/product_shell/claude-3-7-sonnet-latest.py
